https://towardsdatascience.com/six-spark-exercises-to-rule-them-all-242445b24565

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise1") \
    .getOrCreate()

This exercise use below datasets:
1. sales_parquet:
   
        Each row in this table is an order and every order can contain only one product. Each row stores the following fields:
        order_id: The order ID
        product_id: The single product sold in the order. All orders have exactly one product)
        seller_id: The selling employee ID that sold the product
        num_pieces_sold: The number of units sold for the specific product in the order
        bill_raw_text: A string that represents the raw text of the bill associated with the order
        date: The date of the order.

2. Product_parquet:
   
        Each row represents a distinct product. The fields are:
        product_id: The product ID
        product_name: The product name
        price: The product price

3. Seller_parquet:
   
        This table contains the list of all the sellers:
        seller_id: The seller ID
        seller_name: The seller name
        daily_target: The number of items (regardless of the product type) that the seller needs to hit his/her quota. For example, if the daily target is 100,000, the employee needs to sell 100,000 products he can hit the quota by selling 100,000 units of product_0, but also selling 30,000 units of product_1 and 70,000 units of product_2

### How many distinct products have been sold in each day?


In [6]:

#   Read the source tables in Parquet format
products_table = spark.read.parquet("./data/products_parquet")
sales_table = spark.read.parquet("./data/sales_parquet")
sellers_table = spark.read.parquet("./data/sellers_parquet")

In [8]:
products_table.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)



In [9]:
sales_table.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)



In [10]:
sellers_table.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



![picture 1](../images/3c4a71d13ae38c36f9f0901c2cec18a6f2c8fc4cc208eb673476839ff4a308bf.png)  


In [13]:
sellers_table.select("seller_id").distinct().count()
print("Number of seller in sales table", str( sales_table.select("seller_id").distinct().count()))

Number of seller in sales table 10


In [30]:
#sales_table.select("product_id","date").show(10,False)

sales_table.groupBy("date").agg(countDistinct("product_id").alias("distinct_products_sold")).orderBy(col("distinct_products_sold").desc()).show()

+----------+----------------------+
|      date|distinct_products_sold|
+----------+----------------------+
|2020-07-06|                100765|
|2020-07-09|                100501|
|2020-07-01|                100337|
|2020-07-03|                100017|
|2020-07-02|                 99807|
|2020-07-05|                 99796|
|2020-07-04|                 99791|
|2020-07-07|                 99756|
|2020-07-08|                 99662|
|2020-07-10|                 98973|
+----------+----------------------+



### Find out how many orders, how many products and how many sellers are in the data.

### How many products have been sold at least once? Which is the product contained in more orders?

In [31]:
#  Find out how many orders, how many products and how many sellers are in the data.

#   Print the number of orders
print("Number of Orders: {}".format(sales_table.count()))

#   Print the number of sellers
print("Number of sellers: {}".format(sellers_table.count()))

#   Print the number of products
print("Number of products: {}".format(products_table.count()))

Number of Orders: 20000040
Number of sellers: 10
Number of products: 75000000


In [32]:
#   Output how many products have been actually sold at least once
print("Number of products sold at least once")
sales_table.agg(countDistinct(col("product_id"))).show()

Number of products sold at least once
+-----------------+
|count(product_id)|
+-----------------+
|           993429|
+-----------------+



In [33]:

#   Output which is the product that has been sold in more orders
print("Product present in more orders")
sales_table.groupBy(col("product_id")).agg(
    count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(1).show()

Product present in more orders
+----------+--------+
|product_id|     cnt|
+----------+--------+
|         0|19000000|
+----------+--------+



### What is the average revenue of the orders?


### Who are the second most selling and the least selling persons (sellers) for each product? Who are those for product with `product_id = 0`

![picture 1](../images/3c4a71d13ae38c36f9f0901c2cec18a6f2c8fc4cc208eb673476839ff4a308bf.png)  


In [35]:
# We get the sum of sales for each product and seller pairs.

sales_table = sales_table.groupby(col("product_id"), col("seller_id")). \
    agg(sum("num_pieces_sold").alias("num_pieces_sold"))
    
sales_table.count()

999276

In [36]:
# Create the window functions, one will sort ascending the other one descending. Partition by the product_id
# and sort by the pieces sold

# So in question whichever has been mentioned as each we have to partition on that
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType
window_desc = Window.partitionBy(col("product_id")).orderBy(col("num_pieces_sold").desc())
window_asc = Window.partitionBy(col("product_id")).orderBy(col("num_pieces_sold").asc())

In [37]:
# Create a Dense Rank (to avoid holes)

sales_table = sales_table.withColumn("rank_asc", dense_rank().over(window_asc)). \
    withColumn("rank_desc", dense_rank().over(window_desc))


+----------+---------+---------------+--------+---------+
|product_id|seller_id|num_pieces_sold|rank_asc|rank_desc|
+----------+---------+---------------+--------+---------+
|  10005243|        6|           98.0|       1|        1|
|  10023464|        9|           59.0|       1|        1|
|  10050363|        6|           18.0|       1|        1|
|  10089524|        2|           53.0|       1|        1|
|  10122266|        2|           25.0|       1|        1|
|  10134574|        3|           28.0|       1|        1|
|  10150439|        4|           72.0|       1|        1|
|  10158822|        9|           86.0|       1|        1|
|  10160884|        7|           89.0|       1|        1|
|  10172594|        2|           60.0|       1|        1|
|  10175294|        7|           29.0|       1|        1|
|   1017716|        9|           82.0|       1|        1|
|  10200802|        2|           42.0|       1|        1|
|  10215353|        1|           22.0|       1|        1|
|  10218345|  

In [45]:
sales_table.show(50,False)

+----------+---------+---------------+--------+---------+
|product_id|seller_id|num_pieces_sold|rank_asc|rank_desc|
+----------+---------+---------------+--------+---------+
|10005243  |6        |98.0           |1       |1        |
|10023464  |9        |59.0           |1       |1        |
|10050363  |6        |18.0           |1       |1        |
|10089524  |2        |53.0           |1       |1        |
|10122266  |2        |25.0           |1       |1        |
|10134574  |3        |28.0           |1       |1        |
|10150439  |4        |72.0           |1       |1        |
|10158822  |9        |86.0           |1       |1        |
|10160884  |7        |89.0           |1       |1        |
|10172594  |2        |60.0           |1       |1        |
|10175294  |7        |29.0           |1       |1        |
|1017716   |9        |82.0           |1       |1        |
|10200802  |2        |42.0           |1       |1        |
|10215353  |1        |22.0           |1       |1        |
|10218345  |4 

In [41]:
# Get products that only have one row OR the products in which multiple sellers sold the same amount
# (i.e. all the employees that ever sold the product, sold the same exact amount)
single_seller = sales_table.where(col("rank_asc") == col("rank_desc")).select(
    col("product_id").alias("single_seller_product_id"), col("seller_id").alias("single_seller_seller_id"),
    lit("Only seller or multiple sellers with the same results").alias("type"))

In [44]:
single_seller.show(truncate= False)

+------------------------+-----------------------+-----------------------------------------------------+
|single_seller_product_id|single_seller_seller_id|type                                                 |
+------------------------+-----------------------+-----------------------------------------------------+
|10005243                |6                      |Only seller or multiple sellers with the same results|
|10023464                |9                      |Only seller or multiple sellers with the same results|
|10050363                |6                      |Only seller or multiple sellers with the same results|
|10089524                |2                      |Only seller or multiple sellers with the same results|
|10122266                |2                      |Only seller or multiple sellers with the same results|
|10134574                |3                      |Only seller or multiple sellers with the same results|
|10150439                |4                      |Only 

In [46]:
# Get the second top sellers
second_seller = sales_table.where(col("rank_desc") == 2).select(
    col("product_id").alias("second_seller_product_id"), col("seller_id").alias("second_seller_seller_id"),
    lit("Second top seller").alias("type")
)


In [47]:
# Get the least sellers and exclude those rows that are already included in the first piece
# We also exclude the "second top sellers" that are also "least sellers"
least_seller = sales_table.where(col("rank_asc") == 1).select(
    col("product_id"), col("seller_id"),
    lit("Least Seller").alias("type")
).join(single_seller, (sales_table["seller_id"] == single_seller["single_seller_seller_id"]) & (
        sales_table["product_id"] == single_seller["single_seller_product_id"]), "left_anti"). \
    join(second_seller, (sales_table["seller_id"] == second_seller["second_seller_seller_id"]) & (
        sales_table["product_id"] == second_seller["second_seller_product_id"]), "left_anti")


In [48]:
# Union all the pieces
union_table = least_seller.select(
    col("product_id"),
    col("seller_id"),
    col("type")
).union(second_seller.select(
    col("second_seller_product_id").alias("product_id"),
    col("second_seller_seller_id").alias("seller_id"),
    col("type")
)).union(single_seller.select(
    col("single_seller_product_id").alias("product_id"),
    col("single_seller_seller_id").alias("seller_id"),
    col("type")
))
union_table.show()

+----------+---------+------------+
|product_id|seller_id|        type|
+----------+---------+------------+
|  19986717|        1|Least Seller|
|  40496308|        5|Least Seller|
|  52606213|        7|Least Seller|
|  14542470|        5|Least Seller|
|  28592106|        5|Least Seller|
|  17944574|        8|Least Seller|
|  61475460|        7|Least Seller|
|   3534470|        3|Least Seller|
|  35669461|        4|Least Seller|
|  32602520|        9|Least Seller|
|  72017876|        1|Least Seller|
|  67723231|        5|Least Seller|
|  56011040|        5|Least Seller|
|  34681047|        5|Least Seller|
|  57735075|        9|Least Seller|
|  18182299|        7|Least Seller|
|  69790381|        5|Least Seller|
|  31136332|        9|Least Seller|
|  10978356|        7|Least Seller|
|  20774718|        9|Least Seller|
+----------+---------+------------+
only showing top 20 rows



In [49]:
# Which are the second top seller and least seller of product 0?
union_table.where(col("product_id") == 0).show()

+----------+---------+--------------------+
|product_id|seller_id|                type|
+----------+---------+--------------------+
|         0|        0|Only seller or mu...|
+----------+---------+--------------------+

